<span style="font-family:Times; font-size:3em; font-weight:bold">
Python Practice with `fsolve` and `quad`
</span><br/>

---

In [23]:
import numpy as np
from scipy.optimize import fsolve
from scipy.integrate import quad
from myengineerpkg import molecular_weights as mw

molecular_weights is starting import
import completed


# Properties of Ethanol from DIPPR

In [2]:
ethanolMW=46.07      # molecular weight of ethanol in g/mol
ethanolTC=514        # critical temperature of ethanol in K
ethanolPC=61.37e5    # critical pressure of ethanol in Pa
ethanolACEN=0.643558 # acentric factor of ethanol (unitless)

def ethanolICP(t):
    '''
    Input temperature in K
    Return IDEAL GAS heat capacity of ethanol in J/mol/K
    '''
    A=49200; B=145770; C=1662.8; D=93900; E=744.7
    cp=A + B*((C/t)/np.sinh(C/t))**2 + D*((E/t)/np.cosh(E/t))**2  # J/kmol/K
    return(cp/1000) # J/mol/K

def ethanolLCP(t):
    '''
    Input temperature in K
    Return LIQUID heat capacity of ethanol in J/mol/K
    '''
    A=102640; B=-139.63; C=-0.030341; D=0.0020386; E=0
    cp=A + B*t + C*t**2 + D*t**3 + E*t**4 # J/kmol/K
    return(cp/1000) # J/mol/K

def ethanolLDN(t):
    '''
    Input temperature in K
    Return LIQUID density of ethanol in mol/m**3
    '''
    A=1.6288; B=0.27469; C=514; D=0.23178
    rho=A/(B**(1+(1-t/C)**D)) # kmol/m**3
    return(rho*1000) # mol/m**3

def ethanolVP(t):
    '''
    Input temperature in K
    Return LIQUID vapor pressure of ethanol in Pa
    '''
    A=73.304; B=-7122.3; C=-7.1424; D=2.8853E-06; E=2
    lnVP=A + B/t + C*np.log(t) + D*t**E
    return(np.exp(lnVP)) # Pa

def ethanolHVP(t):
    '''
    Input temperature in K
    Return heat of vaporization of ethanol in J/mol
    '''
    tr=t/ethanolTC
    A=65831000; B=1.1905; C=-1.7666; D=1.0012
    hvp=A*(1-tr)**(B + C*tr + D*tr**2) # J/kmol
    return(hvp/1000) # J/mol

# Practice Integration 1
**Question**
One kilogram of liquid *ethanol* in a piston/cylinder apparatus is heated from 300 K to 350 K in a beaker under constant pressure. How much heat is needed?

### Solution Approch
The energy balance for a closed system is
$$\Delta U=Q + W$$
There is not shaft work, and mass is constant, so 
$$m\Delta u=Q$$
Assuming the liquid is incompressible, the change in internal energy is
$$\Delta u=\int^{350\text{ K}}_{300\text{ K}}C^{\text{liq}}_vdT\cong\int^{350\text{ K}}_{300\text{ K}}C^{\text{liq}}_pdT$$

### Numerical Solution
**Knowns**

In [3]:
m=1        # mass of ethanol in kg
T1=300     # initial temperature in K
T2=350     # final temperature in K

**Perform the integration**

In [8]:
Δu=quad(ethanolLCP,T1,T2)[0]   # fill in here
Δu #J/mol

6222.228354166667

**Fix the units**

In [21]:
n=m*1000/ethanolMW  # fill in here
n #mol

21.706099413935316

**Calculate and print the solution**

In [20]:
Q=n*Δu
print(f"Q = {n*Δu:.1f} J")

Q = 135060.3 J


# Practice Integration 2
**Question**
One kilogram of liquid *1-propanol* in a piston/cylinder apparatus is heated from 300 K to 350 K in a beaker under constant pressure. How much heat is needed?

## Solution
**Properties**

In [25]:
def propanolLCP(t):
    '''
    Input temperature in K
    Return LIQUID heat capacity of ethanol in J/mol/K
    '''
    A=158760; B=-635; C=1.969; D=0; E=0
    cp=A + B*t + C*t**2 + D*t**3 + E*t**4 # J/kmol/K
    return(cp/1000) # J/mol/K

propanolMW = mw.by_formula("C3H8O")

**Perform the integration**

In [26]:
Δu=quad(propanolLCP,T1,T2)[0]      # fill in here
Δu #J/mol

8038.541666666667

**Fix the units**

In [27]:
n=m*1000/propanolMW     # fill in here
n #mol

np.float64(16.64004259850905)

**Calculate and print the solution**

In [28]:
Q=n*Δu
print("Q = %.1f J" % Q)

Q = 133761.7 J


# Practice fsolve 1
**Question**
One kilojoule of heat is added to 0.1 kg of vapor *ethanol* initailly at 380 K and 1 atm in a piston/cylinder apparatus. What is the final temperature if the process occurs isobarically?

## Solution Approach
From above, the design equation above,
$$m\Delta u=m\int^{T_\text{final}}_{380\text{ K}}C^{\text{vap}}_vdT=Q$$
Because this is an ideal gas, $C_v^\text{vap}=C_p^\text{vap}-R$ to give
$$m\Delta u=m\int^{T_\text{final}}_{380\text{ K}}\left[C^{\text{vap}}_p-R\right]dT=Q$$
`fsolve` may be used to determine $T_\text{final}$.
## Numerical Solution
**Knowns**

In [61]:
m=0.1                       # mass of ethanol in kg
n=m/(ethanolMW/1000)        # moles of ethanol
T1=380                      # initial temperature in K
Q=1000                      # heat added in J
R=8.314                     # gas constant in J/mol/K

**`fsolve` function**

In [62]:
def LCP(T,coeffs):
    result = 0
    for index in range(len(coeffs)):
        result += coeffs[index]*T**index
    return result

def ICP(T,coeffs):
    A,B,C,D,E = coeffs
    return (A + B*((C/T)/np.sinh(C/T))**2 + D*((E/T)/np.cosh(E/T))**2)/1000

def f(T2):
    return n*quad(lambda T: ICP(T,[49200,145770,1662.8,93900,744.7])-R, T1, T2)[0] - Q

**Define guess values**

In [63]:
guess = 400
f(guess)

2115.4857662570744

**Obtain and print the solutions**

In [64]:
Tf=fsolve(f,guess)[0]      # fill in here
print("Tf = %.1f K" % Tf)

Tf = 386.5 K


# Practice fsolve 2
**Question**
Ethanol(1) and 1-propanol(2) are mixed, and the system is brought to a state where vapor and liquid exist in equilibrium. The pressure is 80 kPa, and the vapor mole fraction of ethanol is measured to be 0.2. What is the temperature of the system, and what is the composition of the liquid?

## Solution Approch
Vapor liquid equilibrium for low pressures and chemically-similar compounds is described by Raoult's Law. (Remember that this class will always use Raoult's Law for VLE. You will learn more accurate approaches later.)
$$x_iP_i^\text{sat}(T)=y_iP$$
One Raoult's Law statement may be written for each component in the system. For the case under consideration, the following is obtained.
$$x_1P_1^\text{sat}(T)=y_1P$$
$$(1-x_1)P_2^\text{sat}(T)=(1-y_1)P$$
The problem statement gave values for $y_1$ and $P$. the remaining unknowns are $x_1$ and $T$. Functions were defined for the vapor pressures as a function of temperature in previous locations in this document.
## Numerical Solution
**Knowns**

In [72]:
x1=0.2       # liquid mole fraction of ethanol
P=80e3       # system pressure in Pa

**Properties**

In [73]:
def VP(T,coeffs):
    A,B,C,D,E = coeffs
    return np.exp(A + B/T + C*np.log(T) + D*T**E)

eth_VP = lambda T: VP(T,[73.304,-7122.3,-7.1424,2.8853E-06,2])
prop_VP = lambda T: VP(T,[84.66416,-8307.24422,-8.57673,7.50905E-18,6])

**`fsolve` function**

In [81]:
def f(variables):
    y1,T = variables
    eq1 = x1*eth_VP(T) - y1*P
    eq2 = (1-x1)*prop_VP(T) - (1-y1)*P
    return [eq1,eq2]

**Define guess values**

In [82]:
guess = np.array([0.5,400])
f(guess)

[np.float64(64508.50203894217), np.float64(183348.64105073246)]

**Obtain and print the solutions**

In [86]:
y1,T = fsolve(f,guess)

In [87]:
print(y1)
print(T)

0.3424373528373853
359.321141932948
